In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization

filname = "../input/fer2013/fer2013/fer2013.csv"
df = pd.read_csv(filname)

In [ ]:
label_map = ['Kızgın', 'İğrenmiş', 'Korkmuş', 'Mutlu', 'Üzgün', 'Şaşkın', 'Kararsız']

data = df["pixels"]

df.head(10)

Elimizdeki veri setini pandas kullanarak import ettik. İçerisindeki pixelleri data değişkenine attık. 
Datasetinin ilk 10 datasını görmek için df.head(10) kullandık.

In [ ]:
def getData(filname):
    # images are 48x48
    # N = 35887
    Y = []
    X = []
    first = True
    for line in open(filname):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])

    X, Y = np.array(X) / 255.0, np.array(Y)
    return X, Y


In [ ]:
X, Y = getData(filname)
num_class = len(set(Y))
print(num_class)

In [ ]:
N, D = X.shape
X = X.reshape(N, 48, 48, 1)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.utils.np_utils import to_categorical

y_categorical = to_categorical(Y)

X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.25, random_state=0)



In [ ]:
def my_model():
    model = Sequential()
    input_shape = (48,48,1)
    model.add(Conv2D(128, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (5, 5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(512, (3, 3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(7))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=Adam(lr=0.001))
    # UNCOMMENT THIS TO VIEW THE ARCHITECTURE
    #model.summary()
    
    return model
model=my_model()
model.summary()

In [ ]:
path_model='model_filter_300_150.h5'
h=model.fit(x=X_train,     
            y=y_train, 
            batch_size=300, 
            epochs=150, 
            verbose=1, 
            validation_data=(X_test,y_test),
            shuffle=True,
            callbacks=[
                ModelCheckpoint(filepath=path_model),
            ]
            )

In [ ]:
import matplotlib.pyplot as plt
f, axarr = plt.subplots(2, sharex=True)
axarr[0].plot(h.history['acc'], color = 'orange')
axarr[0].plot(h.history['val_acc'], color = 'yellow')
axarr[0].legend(['train', 'test'])
axarr[0].set_title('acc - val_acc')
axarr[1].plot(h.history['loss'], color = 'red')
axarr[1].plot(h.history['val_loss'], color = 'gray')
axarr[1].legend(['train', 'test'])
axarr[1].set_title('loss - val_loss')
fig = plt.figure()
fig.savefig("Acc_Graph_400_150.png")
plt.close(fig)
plt.show()

In [ ]:
#from sklearn.metrics import confusion_matrix
#import seaborn as sns

#y_test_non_category = [ np.argmax(t) for t in y_test ]
#y_predict_non_category = [ np.argmax(t) for t in X_test ]
#cm  = confusion_matrix(y_test_non_category, y_predict_non_category)
#f,ax = plt.subplots(figsize=(4, 4))
#sns.heatmap(cm, annot=True, linewidths=0.01,cmap="Oranges",linecolor="Yellow", fmt= '.1f',ax=ax)
#plt.xlabel("Predicted Label")
#plt.ylabel("True Label")
#plt.title("Confusion Matrix")
#fig = pls.figure()
#fig.savefig("Confusion_Matrix_400_150.png")
#plt.close(fig)
#plt.show()*/